In [16]:
from scipy import stats
import statsmodels.api as sm
import pandas as pd
from statsmodels.base.model import GenericLikelihoodModel
import numpy as np

In [14]:
data = sm.datasets.spector.load_pandas()
exog = data.exog
exog = sm.add_constant(exog)

endog = data.endog
print(sm.datasets.spector.NOTE)
print(data.exog.head())

::

    Number of Observations - 32

    Number of Variables - 4

    Variable name definitions::

        Grade - binary variable indicating whether or not a student's grade
                improved.  1 indicates an improvement.
        TUCE  - Test score on economics test
        PSI   - participation in program
        GPA   - Student's grade point average

    GPA  TUCE  PSI
0  2.66  20.0  0.0
1  2.89  22.0  0.0
2  3.28  24.0  0.0
3  2.92  12.0  0.0
4  4.00  21.0  0.0


$$logL(\beta) = \log\left[F(x'\beta)^y\cdot(1-F(x'\beta))^{1-y}\right]$$

In [34]:
class MyProbit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        q = 2 * endog - 1
        return stats.norm.logcdf(q*np.dot(exog, params)).sum()

In [38]:
class ProbitRegression:

    def __init__(self, y, X):
        self.β = np.ones(y.shape)
        self.X, self.y = X, y
        self.n, self.k = X.shape

    def μ(self):
        return norm.cdf(self.X @ self.β.T)

    def ϕ(self):
        return norm.pdf(self.X @ self.β.T)

    def logL(self):
        μ = self.μ()
        return np.sum(y * np.log(μ) + (1 - y) * np.log(1 - μ))

    def G(self):
        μ = self.μ()
        ϕ = self.ϕ()
        return np.sum((X.T * y * ϕ / μ - X.T * (1 - y) * ϕ / (1 - μ)),
                     axis=1)

    def H(self):
        X = self.X
        β = self.β
        μ = self.μ()
        ϕ = self.ϕ()
        a = (ϕ + (X @ β.T) * μ) / μ**2
        b = (ϕ - (X @ β.T) * (1 - μ)) / (1 - μ)**2
        return -(ϕ * (y * a + (1 - y) * b) * X.T) @ X

In [39]:
mod = ProbitRegression(endog,exog)

In [40]:
mod.logL()

NameError: name 'norm' is not defined